### Import modules

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### Practice pandas DataFrame indexing

One way to creata a pandas DataFrame from scratch is to pass it a dictionary as input. Note the recycling of values in columns A and B. Also remember that the columns of the dataframe can be of different types, but the entire column must be of the same type. See in column D how all of the numbers are converted to floats with 2 decimal precision.

In [6]:
df = pd.DataFrame({'A' : 1.,
                   'B' : 1,
                   'C' : ["Steph", "Dylan", "Kate", "Fred", "Mike"],
                   'D' : [3.14, 4, 5, 6, 7]})
df

,A,B,C,D
0,1.0,1,Steph,3.14
1,1.0,1,Dylan,4.00
2,1.0,1,Kate,5.00
3,1.0,1,Fred,6.00
4,1.0,1,Mike,7.00


In [8]:
df.dtypes

A    float64
B      int64
C     object
D    float64
dtype: object

There are several different ways of indexing rows and columns in pandas. The first is integer indexing with the `.iloc[]` method, which extracts rows and/or columns based on their numeric index.

In [9]:
df.iloc[0:2, 0:3]

,A,B,C
0,1.0,1,Steph
1,1.0,1,Dylan


The second is name-based indexing with the `.loc[]` method, which extracts rows and/or columns based on their name.

In [10]:
df.loc[[0, 3], ["A", "C"]]

,A,C
0,1.0,Steph
3,1.0,Fred


The next is a shortcut for name-based indexing of columns by simply enclosing the name of that column in square brackets. Note also that if you extract a single column from a DataFrame, it is returned as a Series--a one-dimensional pandas object.

In [11]:
df["A"]  #series

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: A, dtype: float64

In [13]:
df[["A", "B"]] #dataframe

,A,B
0,1.0,1
1,1.0,1
2,1.0,1
3,1.0,1
4,1.0,1


In contrast, when you extract multiple columns, the subset is returned as another pandas DataFrame.

In [20]:
df[["A", "B"]].shape

(5, 2)

In [21]:
df.shape

(5, 4)

Finally, we have Boolean indexing, where you can extract all rows or columns, again using the `.loc[]` method, where the index is equal to `True`. This is often useful for subsetting rows or columns where the value in that row or column meets some specified condition (e.g., is greater than some number). 

In [19]:
my_bool = [True, False, True, False, True]
df.loc[my_bool]

,A,B,C,D
0,1.0,1,Steph,3.14
2,1.0,1,Kate,5.00
4,1.0,1,Mike,7.00


### Load the GTEx data

Now we can start working with some real data. We obtained human multi-tissue gene expression data from the GTEx Project, by downloading it from the GTEx Data Portal. The file is gzip compressed (ends in the `.gz` extension), so we use the option `compression = "gzip"`. We also skip the first two rows, which have unique formatting and specify that the file is tab-separated.

Download the file here: https://storage.googleapis.com/gtex_analysis_pilot_v3/rna_seq_data/GTEx_Analysis_RNA-seq_RNA-SeQCv1.1.8_gene_rpkm__Pilot_V3_patch1.gct.gz

In [27]:
df_rpkms = pd.read_csv("~/Downloads/RNA-seq.gct.gz",
                       compression = "gzip",
                       skiprows = 2,
                       sep = "\t")

df_rpkms

,Name,Description,GTEX-N7MS-0007-SM-2D7W1,GTEX-N7MS-0011-R10A-SM-2HMJK,GTEX-N7MS-0011-R11A-SM-2HMJS,GTEX-N7MS-0011-R1a-SM-2HMJG,GTEX-N7MS-0011-R2a-SM-2HML6,GTEX-N7MS-0011-R3a-SM-33HC6,GTEX-N7MS-0011-R4a-SM-2HMKW,GTEX-N7MS-0011-R5a-SM-2HMK8,...,GTEX-X4LF-0526-SM-3NMB6,GTEX-X4LF-1726-SM-3NMBZ,GTEX-X4XX-0005-SM-3NMCS,GTEX-X4XX-0011-R1B-SM-3P622,GTEX-X4XX-0011-R2A-SM-3P623,GTEX-X4XX-0126-SM-3NMC2,GTEX-X4XX-0626-SM-3NMC1,GTEX-X4XX-1126-SM-3NMBY,GTEX-X4XX-2926-SM-3NMB1,GTEX-X4XX-3026-SM-3NMB2
0,ENSG00000223972.4,DDX11L1,0.000000,0.000000,0.000000,0.000000,0.000000,0.029181,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050261,0.000000
1,ENSG00000227232.3,WASH7P,2.917592,1.958602,5.841671,1.728239,2.315600,3.742634,2.269886,2.442356,...,9.417033,7.007399,6.276984,3.710413,5.769073,12.540883,4.696292,4.555761,9.459983,5.700174
2,ENSG00000243485.1,MIR1302-11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.102921,0.000000,0.000000,0.000000,0.000000,0.000000
3,ENSG00000237613.2,FAM138A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,ENSG00000240361.1,OR4G11P,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015078,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52571,ENSG00000198786.2,MT-ND5,627.419189,3967.486572,1014.433228,4683.750000,2813.970947,2435.256104,3997.514893,5427.200684,...,3997.743164,1589.340210,277.970520,7192.353516,1723.450684,2418.593018,3559.731689,2795.689209,1894.077515,3302.491699
52572,ENSG00000198695.2,MT-ND6,553.020569,2737.471680,787.444824,3694.725342,1631.243652,2049.827148,2676.096191,3723.490234,...,6915.279297,1867.175537,400.203766,8612.575195,1159.216675,4115.834473,5160.529297,2533.044434,1654.201904,3118.078125
52573,ENSG00000210194.1,J01415.21,0.434417,0.000000,0.243951,0.847667,0.000000,0.357784,1.051482,0.811884,...,2.372880,0.214025,0.000000,0.974938,0.199876,0.621196,1.584828,0.217481,0.410828,0.000000
52574,ENSG00000198727.2,MT-CYB,2022.417969,20763.693359,7525.217285,24014.476562,13874.849609,12816.257812,18059.113281,26712.630859,...,7216.080078,9511.128906,1375.219727,24179.337891,3829.971191,10955.970703,10365.626953,20897.306641,7734.621094,12888.306641


We saw that pandas DataFrames possess some methods that can operate on entire rows or columns. For example, by passing the `axis = 1` option to the `.median()` method, we get the median expression of every gene across all samples.  

In [28]:
df_rpkms.median(axis=1) #median gene expression across columns (axis=1)
                    #1st gene DDX has median expression of 0 across all samples

0           0.000000
1           5.917334
2           0.000000
3           0.000000
4           0.000000
            ...     
52571    2376.176025
52572    2423.041016
52573       0.342601
52574    8809.516602
52575       0.000000
Length: 52576, dtype: float64

In order to make sense of PCA, we need to filter out very low expressed genes. This decision is arbitrary, but here we decided to choose genes with median expression greater than zero. we create a new variable `roi`, which records `True` if median expression is greater than zero and `False` if not.

In [32]:
roi = df_rpkms.median(axis=1) > 0 #give all row where median > 0 
roi #is a series

0        False
1         True
2        False
3        False
4        False
         ...  
52571     True
52572     True
52573     True
52574     True
52575    False
Length: 52576, dtype: bool

We can then subset the genes (stored in rows of the original DataFrame) based on this Boolean Series.

In [34]:
df_rpkms = df_rpkms.loc[roi, :] #GTEX-N7MS is same person, diff tissues. are tissues diff across people?
df_rpkms 

,Name,Description,GTEX-N7MS-0007-SM-2D7W1,GTEX-N7MS-0011-R10A-SM-2HMJK,GTEX-N7MS-0011-R11A-SM-2HMJS,GTEX-N7MS-0011-R1a-SM-2HMJG,GTEX-N7MS-0011-R2a-SM-2HML6,GTEX-N7MS-0011-R3a-SM-33HC6,GTEX-N7MS-0011-R4a-SM-2HMKW,GTEX-N7MS-0011-R5a-SM-2HMK8,...,GTEX-X4LF-0526-SM-3NMB6,GTEX-X4LF-1726-SM-3NMBZ,GTEX-X4XX-0005-SM-3NMCS,GTEX-X4XX-0011-R1B-SM-3P622,GTEX-X4XX-0011-R2A-SM-3P623,GTEX-X4XX-0126-SM-3NMC2,GTEX-X4XX-0626-SM-3NMC1,GTEX-X4XX-1126-SM-3NMBY,GTEX-X4XX-2926-SM-3NMB1,GTEX-X4XX-3026-SM-3NMB2
1,ENSG00000227232.3,WASH7P,2.917592,1.958602,5.841671,1.728239,2.315600,3.742634,2.269886,2.442356,...,9.417033,7.007399,6.276984,3.710413,5.769073,12.540883,4.696292,4.555761,9.459983,5.700174
6,ENSG00000238009.2,RP11-34P13.7,0.177576,0.204927,0.289186,0.248324,0.268730,0.168188,0.408321,0.219865,...,0.229263,0.218716,0.658286,0.314833,0.514728,0.389351,0.255082,0.248918,0.487006,0.290606
7,ENSG00000239945.1,RP11-34P13.8,0.272705,0.202653,0.510465,0.280842,0.316025,0.162210,0.715072,0.254829,...,0.270831,0.134354,0.687998,0.479412,0.533255,0.357458,0.248718,0.273047,0.591012,0.261905
8,ENSG00000237683.3,RP11-34P13.11,88.187675,1.009232,1.577608,0.575865,0.981906,0.532865,1.030279,0.767670,...,10.860014,2.222919,128.481339,2.991933,1.625518,10.951993,0.531858,4.747769,3.747678,2.664726
9,ENSG00000239906.1,RP11-34P13.14,21.158670,0.097359,0.208453,0.000000,0.227738,0.152861,0.000000,0.130077,...,1.013800,0.457205,11.639369,0.208268,0.170792,0.928909,0.126958,0.371670,0.526572,0.171122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52566,ENSG00000198840.2,MT-ND3,546.563416,9898.380859,4572.871582,10415.388672,7144.897461,6341.940430,8621.208984,10017.948242,...,3578.243652,5226.624023,870.697571,13152.982422,3865.071777,4782.514160,3418.035400,16177.149414,4064.408936,6445.033691
52571,ENSG00000198786.2,MT-ND5,627.419189,3967.486572,1014.433228,4683.750000,2813.970947,2435.256104,3997.514893,5427.200684,...,3997.743164,1589.340210,277.970520,7192.353516,1723.450684,2418.593018,3559.731689,2795.689209,1894.077515,3302.491699
52572,ENSG00000198695.2,MT-ND6,553.020569,2737.471680,787.444824,3694.725342,1631.243652,2049.827148,2676.096191,3723.490234,...,6915.279297,1867.175537,400.203766,8612.575195,1159.216675,4115.834473,5160.529297,2533.044434,1654.201904,3118.078125
52573,ENSG00000210194.1,J01415.21,0.434417,0.000000,0.243951,0.847667,0.000000,0.357784,1.051482,0.811884,...,2.372880,0.214025,0.000000,0.974938,0.199876,0.621196,1.584828,0.217481,0.410828,0.000000


The sample names from GTEx (visible as column names here) are not particularly informative. To get more information about the individual GTEx samples, we need to load some separate metadata. This is also available from the GTEx Portal.

Download the file here: https://storage.googleapis.com/gtex_analysis_pilot_v3/annotations/GTEx_Analysis_Annotations_Sample_DS__Pilot_V3.txt

In [36]:
df_metadata = pd.read_csv("~/Downloads/GTEx_Analysis_Annotations_Sample_DS__Pilot_V3.txt", sep = "\t")
df_metadata

,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMTSISCH,SMNABTCH,SMNABTCHT,...,SME1ANTI,SMSPLTRD,SMBSMMRT,SME1SNSE,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS
0,GTEX-N7MS-0007-SM-26GME,NaN,C1,NaN,8.2,Blood,Whole Blood,16-19 hours,BP-16653,RNA isolation_PAXgene Blood RNA (Manual),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GTEX-N7MS-0007-SM-26GMV,NaN,C1,NaN,8.2,Blood,Whole Blood,16-19 hours,BP-16653,RNA isolation_PAXgene Blood RNA (Manual),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GTEX-N7MS-0007-SM-2D43E,NaN,C1,NaN,8.2,Blood,Whole Blood,16-19 hours,BP-16653,RNA isolation_PAXgene Blood RNA (Manual),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GTEX-N7MS-0007-SM-2D7W1,NaN,C1,NaN,8.2,Blood,Whole Blood,16-19 hours,BP-16653,RNA isolation_PAXgene Blood RNA (Manual),...,13772179.0,18422595.0,0.002456,13504096.0,49.508575,0.041526,0.835199,852.0,0.563503,51.355957
4,GTEX-N7MS-0009-SM-2BWY4,NaN,C1,NaN,NaN,Blood,Whole Blood,16-19 hours,BP-16657,DNA isolation_Whole Blood _QIAGEN Puregene (Ma...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3052,K-562-SM-3NMAP,NaN,NaN,NaN,9.5,Bone Marrow,Cells - Leukemia cell line (CML),NaN,BP-17177,RNA isolation_Trizol Manual (Cell Pellet),...,22545176.0,24349348.0,0.003103,22649108.0,50.114986,0.018574,0.936381,897.0,0.284208,50.118595
3053,K-562-SM-3NMDG,NaN,NaN,NaN,9.5,Bone Marrow,Cells - Leukemia cell line (CML),NaN,BP-17177,RNA isolation_Trizol Manual (Cell Pellet),...,20755972.0,22511288.0,0.002659,20865295.0,50.131330,0.017522,0.950120,868.0,0.267642,50.126090
3054,K-562-SM-3P61Y,NaN,NaN,NaN,9.5,Bone Marrow,Cells - Leukemia cell line (CML),NaN,BP-17177,RNA isolation_Trizol Manual (Cell Pellet),...,23599904.0,25524851.0,0.002788,23712745.0,50.119250,0.022186,0.946919,889.0,0.257213,50.105167
3055,NA12878-SM-2XJZN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cell Line DNA (Derived from Blood Cells),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


For the purpose of this exercise, we are really only interested in the tissue from which each sample was obtained. So let's subset the metadata DataFrame to just the sample name and tissue name.

### Principal component analysis

Now we are ready to get started with PCA. This is a method that operates on a numeric matrix, so we need to first remove the character columns (gene ID and gene name) from the DataFrame.

I learned this by trial and error, but it is also necessary to transpose the DataFrame (switch the row/column orientation) so that we are treating the genes as the measured variables rather than the samples. In this case, we are interested in the similarity/difference among samples in their patterns of gene expression. Transposing is achieved using `DataFrame.transpose()` or `DataFrame.T`.

Another practical step prior to PCA is standardizing the input variables (genes, in this case) to have the same means and variances (in expression across samples). This helps give the genes more equal "weight" in PCA. Let's look at the current means and variances.

Then standardize using `StandardScaler().fit_transform()` and check the new means and variances. Notice that the new means are all very close to zero, and the new variances are all equal to 1.

We are finally ready to execute the PCA function! The syntax is a little bit unfamiliar, but here we are specifying that we would like to store the top 10 principal components in `pca_output`.

As expected, our output has rows equal to the number of input samples and columns equal to the number of principal components that we decided to store. We can put this output into a new pandas DataFrame.

And look at where our samples land on the new principal component axes. Here we plot the top two principal components.

The PCA output DataFrame does not note the names of the samples. To get the names, we have to grab them from the row names of our `pca_input` DataFrame.

Now we can merge our PCA output DataFrame with our metadata DataFrame, since they share a column of unique sample IDs. We use the pandas `merge()` function to achieve this.

The last step is to add color to our plot by grouping our DataFrame on tissue, then iterating over the tissues and adding to a scatter plot. The default location of the legend overlaps the plot, so we provide some options to `plt.legend()` to shift it to the right and create two columns. We figured this out by reading the documentation for the corresponding functions and searching online!